In [18]:
# Leitura dos microdados 2020 - Data de atualização: 02/12/2021
import pandas as pd

df = pd.read_csv('./microdados_matriculas_2020.csv', sep=';')

df.head()

,Carga Horaria,Carga Horaria Minima,Categoria de Situação,Cod Unidade,Cor/Raça,Código da Matrícula,Código da Unidade de Ensino - SISTEC,Código do Ciclo de Matrícula,Código do Município com DV,Data de Fim Previsto do Ciclo,...,Vagas Ofertadas,Vagas Regulares AC,Vagas Regulares l1,Vagas Regulares l10,Vagas Regulares l13,Vagas Regulares l14,Vagas Regulares l2,Vagas Regulares l5,Vagas Regulares l6,Vagas Regulares l9
0,3182,800,Concluintes,17,BRANCA,66845709,13791,2016192,5201108,20/12/2018,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3641,1200,Concluintes,279,BRANCA,66801823,3508,2016354,5211503,20/12/2018,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5800,4000,Concluintes,523,BRANCA,64271041,2956,1859227,5221809,21/12/2018,...,40.0,20.0,3.0,1.0,1.0,1.0,5.0,3.0,5.0,1.0
3,4370,3600,Evadidos,523,BRANCA,44608044,2956,1206840,5221809,22/12/2016,...,40.0,20.0,3.0,1.0,1.0,1.0,5.0,3.0,5.0,1.0
4,5800,4000,Concluintes,523,BRANCA,59220389,2956,1859227,5221809,21/12/2018,...,40.0,20.0,3.0,1.0,1.0,1.0,5.0,3.0,5.0,1.0


In [19]:
df.columns.tolist()

['Carga Horaria',
 'Carga Horaria Minima',
 'Categoria de Situação',
 'Cod Unidade',
 'Cor/Raça',
 'Código da Matrícula',
 'Código da Unidade de Ensino - SISTEC',
 'Código do Ciclo de Matrícula',
 'Código do Município com DV',
 'Data de Fim Previsto do Ciclo',
 'Data de Inicio do Ciclo',
 'Data de Ocorrência da Matrícula',
 'Eixo Tecnológico',
 'Faixa Etária',
 'Fator Esforco Curso',
 'Fonte de Financiamento',
 'Idade',
 'Instituição',
 'Matrícula 2019',
 'Modalidade de Ensino',
 'Mês de Ocorrência da Situação',
 'Nome de Curso',
 'Número de registros',
 'Região',
 'Renda Familiar',
 'Sexo',
 'Situação de Matrícula',
 'Subeixo Tecnológico',
 'Tipo de Curso',
 'Tipo de Oferta',
 'Tipo de Unidade',
 'Tipo Oferta',
 'Turno',
 'UF',
 'Unidade de Ensino',
 'Vagas Extraordinárias AC',
 'Vagas Extraordinárias l1',
 'Vagas Extraordinárias l10',
 'Vagas Extraordinárias l13',
 'Vagas Extraordinárias l14',
 'Vagas Extraordinárias l2',
 'Vagas Extraordinárias l5',
 'Vagas Extraordinárias l6',
 'Va

In [21]:
# Filtrando dados pela instituição IFSUL
df['dt_ocorrencia_matricula'] = pd.to_datetime(df['Data de Ocorrência da Matrícula'], format='%d/%m/%Y')
df['ano_ocorrencia_matricula'] = df['dt_ocorrencia_matricula'].dt.year
df['dt_fim_ciclo'] = pd.to_datetime(df['Data de Fim Previsto do Ciclo'], format='%d/%m/%Y')
df['ano_fim_ciclo'] = df['dt_fim_ciclo'].dt.year

qtd_matriculas = df[(df['Instituição'] == 'IFSUL')]
qtd_matriculas.shape

(46349, 58)

In [73]:
# Quantidade de estudantes por curso do campus IFSUL
qtd_estudantes = qtd_matriculas.groupby(['Nome de Curso'])['Código da Matrícula'].count()

print("A média de estudantes por curso é: {:.2f}".format(qtd_estudantes.mean()))
print("O desvio padrão de estudantes por curso é: {:.2f}".format(qtd_estudantes.std()))
print("A moda é:", int(qtd_estudantes.mode()))
print("A mediana de estudantes por curso é: {:.2f}".format(qtd_estudantes.median()))

A média de estudantes por curso é: 565.23
O desvio padrão de estudantes por curso é: 1150.50
A moda é: 171
A mediana de estudantes por curso é: 211.00


In [150]:
# Quantidade de cursos da EJA e do CTI
filtro_cti = (qtd_matriculas['Tipo de Oferta'] == 'Integrado') 
filtro_eja = (qtd_matriculas['Tipo de Oferta'] == 'PROEJA - Integrado') 

qtd_cursos_cti = qtd_matriculas[filtro_cti].groupby(['Tipo de Oferta', 'Nome de Curso'])['Tipo de Oferta'].count().sum()
qtd_cursos_eja = qtd_matriculas[filtro_eja].groupby(['Tipo de Oferta', 'Nome de Curso'])['Tipo de Oferta'].count().sum()
                        
total = qtd_cursos_cti + qtd_cursos_eja
print("Quantidade total de cursos EJA e CTI:", total)

Quantidade total de cursos EJA e CTI: 8726


In [221]:
# Quantidade de estudantes evadidos em cursos superiores do campus IFSUL
filtro_evadidos_licenciatura = qtd_matriculas[
            (qtd_matriculas['Situação de Matrícula'] == 'Abandono')
            & (qtd_matriculas['Tipo de Curso'] == 'Licenciatura')
        ]

evadidos_licenciatura = filtro_evadidos_licenciatura.groupby(['Situação de Matrícula', 'Tipo de Curso'])['Situação de Matrícula'].count()


filtro_evadidos_bacharelado = qtd_matriculas[
            (qtd_matriculas['Situação de Matrícula'] == 'Abandono')
            & (qtd_matriculas['Tipo de Curso'] == 'Bacharelado')
        ]

evadidos_bacharelado = filtro_evadidos_bacharelado.groupby(['Situação de Matrícula', 'Tipo de Curso'])['Situação de Matrícula'].count()

total_evadidos = evadidos_licenciatura.sum() + evadidos_bacharelado.sum()

print("Quantidade de estudantes evadidos em cursos superiores do campus IFSUL é:", total_evadidos)

Quantidade de estudantes evadidos em cursos superiores do campus IFSUL é: 97


In [225]:
# Totalização da classificação racial para cursos EJA
filtro = qtd_matriculas[
    (qtd_matriculas['Tipo de Oferta'] == 'PROEJA - Integrado')
]

filtro.groupby(['Tipo de Oferta', 'Cor/Raça'])['Tipo de Oferta'].count()

Tipo de Oferta      Cor/Raça     
PROEJA - Integrado  AMARELA            2
                    BRANCA           399
                    INDÍGENA           1
                    NÃO DECLARADA    477
                    PARDA             83
                    PRETA             66
Name: Tipo de Oferta, dtype: int64

In [226]:
# Faixa de renda per capita familiar do aluno, declarada na PNP pela instituição
filtro = qtd_matriculas[
    (qtd_matriculas['Tipo de Oferta'] != 'Não se aplica')
    & (qtd_matriculas['Tipo de Oferta'] != 'Concomitante')
    & (qtd_matriculas['Tipo de Oferta'] != 'Subsequente')
]

filtro_tipo_oferta = filtro.groupby(['Tipo de Oferta','Renda Familiar'])['Renda Familiar'].count()
filtro_tipo_oferta


Tipo de Oferta      Renda Familiar
Integrado           0,5<RFP<=1         945
                    0<RFP<=0,5         557
                    1,5<RFP<=2,5       588
                    1<RFP<=1,5         749
                    2,5<RFP<=3,5       259
                    Não declarada     4298
                    RFP>3,5            302
PROEJA - Integrado  0,5<RFP<=1         142
                    0<RFP<=0,5         151
                    1,5<RFP<=2,5        53
                    1<RFP<=1,5          76
                    2,5<RFP<=3,5        23
                    Não declarada      557
                    RFP>3,5             26
Name: Renda Familiar, dtype: int64

In [239]:
# Quantidade de matrículas no curso Análise e Desenvolvimento de Sistemas
f = qtd_matriculas[
    (qtd_matriculas['Nome de Curso']) == 'Análise e Desenvolvimento de Sistemas'
]

qtd = f.groupby(['Nome de Curso', 'Instituição'])['Nome de Curso'].count().sum()
print('Quantidade de matrículas no curso Análise e Desenvolvimento de Sistemas:', qtd)

Quantidade de matrículas no curso Análise e Desenvolvimento de Sistemas: 369


In [272]:
# Média, Desvio padrão, Mediana das quantidades de matriculados em cursos do tipo Técnico
filtro_cursos_técnicos = qtd_matriculas[(qtd_matriculas['Tipo de Curso'] == 'Técnico')]

cursos = filtro_cursos_técnicos.groupby(['Nome de Curso'])['Nome de Curso'].count()

print("A média de matriculados é: {:.2f}".format(cursos.mean()))
print("O desvio padrão de matriculados é: {:.2f}".format(cursos.std()))
print("A mediana de matriculados é: {:.2f}".format(cursos.median()))

A média de matriculados é: 331.00
O desvio padrão de matriculados é: 317.35
A mediana de matriculados é: 225.00


In [276]:
# Filtros classificação racial CTI e EJA - IFSUL
f_integrado = qtd_matriculas[(qtd_matriculas['Tipo Oferta'] == 'Integrado')]
qtd_matriculas_integrado = f_integrado.groupby(['Tipo Oferta', 'Situação de Matrícula'])['Tipo Oferta'].count().sum()

In [280]:
# Filtro classificação racial - CTI - amarela
filtro_integrado_amarela = f_integrado[(f_integrado['Cor/Raça']=='AMARELA')]
amarela = filtro_integrado_amarela.groupby(['Tipo Oferta','Cor/Raça'])['Cor/Raça'].count().sum()
print("Matriculas de classificação racial amarela", amarela)

Matriculas de classificação racial amarela 18


In [281]:
# Filtro classificação racial - CTI - indígena
filtro_integrado_indigena = f_integrado[(f_integrado['Cor/Raça']=='INDÍGENA')]
indigena = filtro_integrado_indigena.groupby(['Tipo Oferta','Cor/Raça'])['Cor/Raça'].count().sum()
print("Matriculas de classificação racial indigena", indigena)

Matriculas de classificação racial indigena 13


In [284]:
# Filtro classificação racial - CTI - não declarada
filtro_integrado_nao_declarada = f_integrado[(f_integrado['Cor/Raça']=='NÃO DECLARADA')]
nao_declarada = filtro_integrado_nao_declarada.groupby(['Tipo Oferta','Cor/Raça'])['Cor/Raça'].count().sum()
print("Matriculas de classificação racial não declarada", nao_declarada)

Matriculas de classificação racial não declarada 3331
